# gridfinder
Run through the full gridfinder model from data input to final guess for Burundi.
Note that the 'truth' data used for the grid here is very bad, so the accuracy results don't mean much.

In [ ]:
import os
from pathlib import Path

import folium
import geopandas as gpd
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import seaborn as sns
from IPython.display import Markdown, display
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

import gridfinder as gf
from gridfinder import save_raster

## Set folders and parameters

In [ ]:
data_in = Path("../tests/data")
aoi_in = data_in / "aoi.geojson"
roads_in = data_in / "roads.geojson"
pop_in = data_in / "pop.tif"
ntl_in = data_in / "ntl.tif"
grid_truth_in = data_in / "grid.geojson"

data_out = Path(".")
targets_out = data_out / "targets.tif"
targets_clean_out = data_out / "targets_clean.tif"
costs_out = data_out / "costs.tif"

dist_out = data_out / "dist.tif"
guess_out = data_out / "guess.tif"
guess_thin_out = data_out / "guess_thin.tif"
guess_vec_out = data_out / "guess.gpkg"

## Clip, filter and resample NTL

In [ ]:
ntl_thresh, affine = gf.prepare_ntl(ntl_in, aoi_in)
save_raster(targets_out, ntl_thresh, affine)
print("Targets prepared")
plt.imshow(ntl_thresh, cmap="viridis")

## Remove target areas with no underlying population

In [ ]:
targets_clean = gf.drop_zero_pop(targets_out, pop_in, aoi_in)
save_raster(targets_clean_out, targets_clean, affine)
print("Removed zero pop")
plt.imshow(ntl_thresh, cmap="viridis")

## Roads: assign values, clip and rasterize

In [ ]:
roads_raster, affine = gf.prepare_roads(roads_in, aoi_in, targets_out)
save_raster(costs_out, roads_raster, affine, nodata=-1)
print("Costs prepared")
plt.imshow(roads_raster, cmap="viridis", vmin=0, vmax=1)

## Get targets and costs and run algorithm

In [ ]:
targets, costs, start, affine = gf.get_targets_costs(targets_clean_out, costs_out)
est_mem = gf.estimate_mem_use(targets, costs)
print(f"Estimated memory usage: {est_mem:.2f} GB")

In [ ]:
dist = gf.optimise(targets, costs, start)
save_raster(dist_out, dist, affine)
plt.imshow(dist)

## Filter dist results to grid guess

In [ ]:
guess, affine = gf.threshold(dist_out, cutoff=0.0)
save_raster(guess_out, guess, affine)
print("Got guess")
plt.imshow(guess, cmap="viridis")

## Check results

In [ ]:
true_pos, false_neg = gf.accuracy(grid_truth_in, guess_out, aoi_in)
print(f"Points identified as grid that are grid: {100*true_pos:.0f}%")
print(f"Actual grid that was missed: {100*false_neg:.0f}%")

## Skeletonize

In [ ]:
guess_skel, affine = gf.thin(guess_out)
save_raster(guess_thin_out, guess_skel, affine)
print("Skeletonized")
plt.imshow(guess_skel)

## Convert to geometry

In [ ]:
guess_gdf = gf.raster_to_lines(guess_thin_out)
guess_gdf.to_file(guess_vec_out, driver="GPKG")
print("Converted to geom")

In [ ]:
minx, miny, maxx, maxy = list(guess_gdf.bounds.iloc[0])
bounds = ((miny, minx), (maxy, maxx))

m = folium.Map(control_scale=True)
m.fit_bounds(bounds)
folium.GeoJson(guess_gdf).add_to(m)
m